In [1]:
from selenium import webdriver

In [2]:
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [47]:
driver=webdriver.Chrome()

                **************** Scraping Data for Dubai *************

In [29]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQyIAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-782831&nflt=ht_id%3D204&offset=0')

In [30]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [31]:
hotel_urls

['https://www.booking.com/hotel/ae/luxury-residence-in-5-star.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQyIAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&ucfs=1&arphpl=1&dest_id=-782831&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&nflt=ht_id%3D204&srpvid=383238c63e360190&srepoch=1697875481&from=searchresults#hotelTmpl',
 'https://www.booking.com/hotel/ae/resivation-al-furjan.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQyIAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&ucfs=1&arphpl=1&dest_id=-782831&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&nflt=ht_id%3D204&srpvid=383238c63e360190&srepoch=1697875481&from_sustainabl

In [32]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Dubai = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Dubai)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [34]:
df_Dubai.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Luxury residence in 5 star hotel,yes,yes,yes,no,yes,no,no,None,None,None
1,Resivation Hotel,yes,no,yes,yes,no,no,no,Continental,Full English/Irish,Vegetarian
2,ibis Styles Dubai Airport Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Vegan
3,Millennium Place Barsha Heights Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Halal
4,IntercityHotel Dubai Jaddaf Waterfront,yes,yes,yes,no,no,no,no,Continental,Gluten-free,Buffet


                ********************Scraping Data for London *********************

In [35]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQyIAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-2601889&nflt=ht_id%3D204')

In [36]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [37]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_london = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_london)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [46]:
df_london.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,The Quarters Swiss Cottage,no,no,yes,yes,no,yes,yes,None,None,None
1,Elsham Apartments,no,yes,yes,no,no,no,no,None,None,None
2,Hyatt Regency London Stratford,yes,yes,yes,yes,yes,no,no,Continental,Full English/Irish,Vegetarian
3,Hotel Saint London,yes,no,yes,yes,no,yes,yes,Continental,Full English/Irish,Vegetarian
4,Hampton by Hilton London Park Royal,yes,yes,yes,yes,yes,no,no,Continental,Full English/Irish,Vegetarian


In [40]:
combined_df = pd.concat([df_Dubai, df_london], ignore_index=True)

In [41]:
combined_df

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Luxury residence in 5 star hotel,yes,yes,yes,no,yes,no,no,None,None,None
1,Resivation Hotel,yes,no,yes,yes,no,no,no,Continental,Full English/Irish,Vegetarian
2,ibis Styles Dubai Airport Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Vegan
3,Millennium Place Barsha Heights Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Halal
4,IntercityHotel Dubai Jaddaf Waterfront,yes,yes,yes,no,no,no,no,Continental,Gluten-free,Buffet
...,...,...,...,...,...,...,...,...,...,...,...
395,Fitzrovia Hotel,yes,yes,yes,yes,yes,no,yes,Continental,None,None
396,Room Home Stay,yes,yes,yes,no,yes,no,no,None,None,None
397,The Portico Hotel,yes,yes,yes,no,no,yes,no,None,None,None
398,Sun Street Hotel Shoreditch,yes,no,yes,yes,yes,no,yes,Continental,Full English/Irish,Vegetarian


In [42]:
combined_df.to_csv('csv2.csv', index=False)

                ****************** Scraping for City - Kuala Lumpur, Malaysia **************************

In [48]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQyIAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-2403010&nflt=ht_id%3D204')

In [49]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [50]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Malaysia = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Malaysia)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [52]:
df_Malaysia.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,ViiA Residences Kuala Lumpur,yes,yes,yes,no,no,no,no,None,None,None
1,OYO 90865 Hotel De Nice Inn,no,no,yes,no,no,no,no,None,None,None
2,"Millerz Square Mid Valley, Old Klang Road",yes,yes,yes,no,no,no,yes,None,None,None
3,Chambers Residence Kuala Lumpur by Roam,yes,no,yes,no,no,no,yes,None,None,None
4,"Tribeca Residences Kuala Lumpur, Five Senses",yes,no,yes,yes,yes,no,no,None,None,None


In [59]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia], ignore_index=True)

In [60]:
combined_df

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Luxury residence in 5 star hotel,yes,yes,yes,no,yes,no,no,None,None,None
1,Resivation Hotel,yes,no,yes,yes,no,no,no,Continental,Full English/Irish,Vegetarian
2,ibis Styles Dubai Airport Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Vegan
3,Millennium Place Barsha Heights Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Halal
4,IntercityHotel Dubai Jaddaf Waterfront,yes,yes,yes,no,no,no,no,Continental,Gluten-free,Buffet
...,...,...,...,...,...,...,...,...,...,...,...
595,Hotel Mezzo,yes,yes,yes,no,no,no,no,None,None,None
596,Hotel Caliber,yes,yes,yes,no,no,no,yes,None,None,None
597,Pullman Kuala Lumpur City Centre Hotel & Resid...,yes,yes,yes,no,yes,no,no,None,None,None
598,Expressionz Premium Suites @ KLCC,no,yes,yes,no,no,no,yes,None,None,None


In [61]:
combined_df.to_csv('csv2.csv', index=False)

        *************************** Webscraping for city - Manchester *****************************

In [57]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-2602512&nflt=ht_id%3D204')

In [58]:
hotel_urls = []
start = 1
end = 5

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [62]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Manchester = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Manchester)

Processed 1/100 URLs
Processed 2/100 URLs
Processed 3/100 URLs
Processed 4/100 URLs
Processed 5/100 URLs
Processed 6/100 URLs
Processed 7/100 URLs
Processed 8/100 URLs
Processed 9/100 URLs
Processed 10/100 URLs
Processed 11/100 URLs
Processed 12/100 URLs
Processed 13/100 URLs
Processed 14/100 URLs
Processed 15/100 URLs
Processed 16/100 URLs
Processed 17/100 URLs
Processed 18/100 URLs
Processed 19/100 URLs
Processed 20/100 URLs
Processed 21/100 URLs
Processed 22/100 URLs
Processed 23/100 URLs
Processed 24/100 URLs
Processed 25/100 URLs
Processed 26/100 URLs
Processed 27/100 URLs
Processed 28/100 URLs
Processed 29/100 URLs
Processed 30/100 URLs
Processed 31/100 URLs
Processed 32/100 URLs
Processed 33/100 URLs
Processed 34/100 URLs
Processed 35/100 URLs
Processed 36/100 URLs
Processed 37/100 URLs
Processed 38/100 URLs
Processed 39/100 URLs
Processed 40/100 URLs
Processed 41/100 URLs
Processed 42/100 URLs
Processed 43/100 URLs
Processed 44/100 URLs
Processed 45/100 URLs
Processed 46/100 UR

In [64]:
df_Manchester.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Hampton By Hilton Manchester Northern Quarter,yes,no,yes,yes,yes,no,no,Continental,Full English/Irish,Vegetarian
1,ibis budget Manchester Salford Quays,yes,yes,yes,no,no,no,no,Continental,Gluten-free,None
2,Motel One Manchester-St. Peter´s Square,yes,no,yes,no,yes,no,no,Continental,None,None
3,Motel One Manchester-Royal Exchange,yes,no,yes,no,yes,no,no,Continental,None,None
4,Motel One Manchester-Piccadilly,yes,no,yes,no,yes,no,no,Continental,None,None


In [65]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester], ignore_index=True)

In [66]:
combined_df

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Luxury residence in 5 star hotel,yes,yes,yes,no,yes,no,no,None,None,None
1,Resivation Hotel,yes,no,yes,yes,no,no,no,Continental,Full English/Irish,Vegetarian
2,ibis Styles Dubai Airport Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Vegan
3,Millennium Place Barsha Heights Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Halal
4,IntercityHotel Dubai Jaddaf Waterfront,yes,yes,yes,no,no,no,no,Continental,Gluten-free,Buffet
...,...,...,...,...,...,...,...,...,...,...,...
695,The Westlynne Hotel & Apartments,yes,yes,yes,no,yes,no,no,Full English/Irish,None,None
696,Whitworth Locke,yes,yes,yes,yes,yes,no,no,None,None,None
697,"Trafford Hall Hotel Manchester, Trademark Coll...",yes,yes,yes,yes,yes,no,no,Full English/Irish,Vegetarian,Vegan
698,"The Waterside Bar, Rooms & Restaurant",yes,no,yes,yes,yes,no,no,None,None,None


In [67]:
combined_df.to_csv('csv2.csv', index=False)

                     *********** Webscraping for City - New Delhi***************

In [71]:
driver=webdriver.Chrome()

In [72]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=en-in-booking-desktop-SoQWfYhAMBURf0HSQntj1AS652796016141%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-334108349%3Alp9302961%3Ali%3Adec%3Adm&sid=b8f957b39c045a637fc32a3a001d95d0&aid=2311236&city=-2106102&nflt=ht_id%3D204')

In [73]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [74]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Delhi = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Delhi)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [78]:
df_Delhi.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Hotel TEj Unit by Flypath Near Delhi Airport,no,yes,yes,no,no,no,no,Continental,None,None
1,Hotel Anchal Dx,yes,no,yes,no,no,no,no,None,None,None
2,OYO Flagship Hotel Yuvraj Paalace,yes,no,no,no,no,no,no,None,None,None
3,Hotel Tripple Tree - Karol Bagh,no,no,yes,no,no,no,no,Continental,Vegetarian,Halal
4,Hotel Grand Suites By D Capitol,yes,no,yes,no,no,no,no,Continental,None,None


In [76]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi], ignore_index=True)

In [77]:
combined_df.to_csv('csv2.csv', index=False)

                *************** Webscraping for city - Birmingham *************************

In [81]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-2589989&nflt=ht_id%3D204')

In [82]:
hotel_urls = []
start = 1
end = 4

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [84]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Birmingham = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Birmingham)

Processed 1/75 URLs
Processed 2/75 URLs
Processed 3/75 URLs
Processed 4/75 URLs
Processed 5/75 URLs
Processed 6/75 URLs
Processed 7/75 URLs
Processed 8/75 URLs
Processed 9/75 URLs
Processed 10/75 URLs
Processed 11/75 URLs
Processed 12/75 URLs
Processed 13/75 URLs
Processed 14/75 URLs
Processed 15/75 URLs
Processed 16/75 URLs
Processed 17/75 URLs
Processed 18/75 URLs
Processed 19/75 URLs
Processed 20/75 URLs
Processed 21/75 URLs
Processed 22/75 URLs
Processed 23/75 URLs
Processed 24/75 URLs
Processed 25/75 URLs
Processed 26/75 URLs
Processed 27/75 URLs
Processed 28/75 URLs
Processed 29/75 URLs
Processed 30/75 URLs
Processed 31/75 URLs
Processed 32/75 URLs
Processed 33/75 URLs
Processed 34/75 URLs
Processed 35/75 URLs
Processed 36/75 URLs
Processed 37/75 URLs
Processed 38/75 URLs
Processed 39/75 URLs
Processed 40/75 URLs
Processed 41/75 URLs
Processed 42/75 URLs
Processed 43/75 URLs
Processed 44/75 URLs
Processed 45/75 URLs
Processed 46/75 URLs
Processed 47/75 URLs
Processed 48/75 URLs
P

In [85]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

In [86]:
df_Birmingham.head()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,The Birmingham Boutique,yes,yes,no,no,no,yes,no,None,None,None
1,Campanile Hotel - Birmingham,no,no,yes,no,yes,yes,no,Full English/Irish,Vegetarian,Gluten-free
2,"Birmingham Strathallan Hotel, BW Signature Col...",no,no,yes,no,no,yes,no,Continental,Full English/Irish,None
3,Castle Bromwich Hall; Sure Hotel Collection by...,no,yes,no,no,no,yes,no,Continental,Full English/Irish,Vegetarian
4,The Bournbrook Inn,no,yes,yes,yes,yes,yes,no,Full English/Irish,Gluten-free,None


                ********************* Webscraping for City - Berlin *******************

In [87]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-1746443&nflt=ht_id%3D204')

In [88]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [89]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Berlin = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Berlin)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [90]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

                ******************** Webscraping for city - Melbourne *********************

In [91]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-1586844&nflt=ht_id%3D204')

In [92]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [93]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Sydney = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Sydney)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [94]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin,
                        df_Sydney], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

                 ********************* Webscraping for city - Sydney **********************

In [95]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-1603135&nflt=ht_id%3D204')

In [96]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [97]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Sydney1 = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Sydney1)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [98]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin,
                        df_Sydney, df_Sydney1], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

                    ******************** Webscraping for city - Paris *******************

In [100]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-1456928&nflt=ht_id%3D204')

In [101]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [102]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Paris = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Paris)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [108]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin,
                        df_Sydney, df_Sydney1, df_Paris], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

                 ********************* Scraping Data for CIty - Tokyo ****************************

In [106]:
driver.get('https://www.booking.com/searchresults.en-gb.html?label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&aid=397646&city=-246227&nflt=ht_id%3D204')

In [107]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [109]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Tokyo = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Tokyo)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [110]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin,
                        df_Sydney, df_Sydney1, df_Paris, df_Tokyo], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

In [123]:
combined_df.tail()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
1970,Shinagawa Prince Hotel,no,no,yes,no,no,no,yes,Buffet,None,None
1971,Hotel Wing International Premium Shibuya,yes,no,yes,yes,no,yes,yes,Asian,American,None
1972,Tokyu Stay Nishi Shinjuku,no,yes,yes,no,no,yes,yes,Continental,Asian,American
1973,Asakusa Kokono Club Hotel,yes,no,yes,no,no,no,yes,Vegetarian,Vegan,Asian
1974,La Vista Tokyo Bay,no,no,no,no,no,no,yes,Continental,Asian,American


                  ********************* Web Scrapping for City - Toronto ****************************

In [124]:
driver.get('https://www.booking.com/searchresults.en-gb.html?aid=397646&label=yho748jc-1FCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AurS96YGwAIB0gIkMTJhN2VkNjAtYzc1MS00MjVhLTg3ZjYtMzI5ZWMzNzlkNzhm2AIG4AIB&sid=b8f957b39c045a637fc32a3a001d95d0&city=-574890&')

In [125]:
hotel_urls = []
start = 1
end = 9

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a78ca197d0"]')
    for i in url:
        hotel_urls.append(i.get_attribute('href'))
    nxt_button = driver.find_element(By.XPATH, '//div[@class="b16a89683f cab1524053"]')
    nxt_button.click()
    time.sleep(2)

In [126]:
import pandas as pd

# Define the amenities you want to check
amenities_to_check = [
    "Free WiFi",
    "Family rooms",
    "Non-smoking rooms",
    "Restaurant",
    "Bar",
    "Heating",
    "Lift"
]

# Create lists for each amenity
amenity_availability = {amenity: [] for amenity in amenities_to_check}
breakfast_cuisines = [[], [], []]  # Three columns for Breakfast Cuisine
hotel_names = []

# Provide a list of all URLs to scrape
url_list = hotel_urls

# Count the number of URLs provided
total_urls_to_scrape = len(url_list)
urls_scraped = 0

# Process the URLs in the provided list
for url in url_list:
    driver.get(url)
    time.sleep(2)

    # Initialize name_text and breakfast_text outside the try block
    name_text = None
    breakfast_text = None

    try:
        # Find the elements for amenities and hotel name
        amenities_element = driver.find_element(By.XPATH, '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]')
        name_elements = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        name_text = name_elements.text

        # Extract the text from the amenities element
        amenities_text = amenities_element.text

        # Check each amenity
        for amenity in amenities_to_check:
            if amenity in amenities_text:
                amenity_availability[amenity].append("yes")
            else:
                amenity_availability[amenity].append("no")

        # Find the element for breakfast, if it exists
        try:
            breakfast_element = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
            breakfast_text = breakfast_element.text
        except NoSuchElementException:
            breakfast_text = None

        # Extract breakfast cuisine values if breakfast_text is not None
        if breakfast_text is not None:
            breakfast_cuisines_values = breakfast_text.split(', ')
        else:
            breakfast_cuisines_values = []

        # Process breakfast_cuisines_values
        for i in range(3):
            if i < len(breakfast_cuisines_values):
                breakfast_cuisines[i].append(breakfast_cuisines_values[i])
            else:
                breakfast_cuisines[i].append(None)

    except NoSuchElementException:
        # If "NoSuchElement" exception occurs, append "No" to amenity lists
        for amenity in amenities_to_check:
            amenity_availability[amenity].append("No")

        # Fill breakfast cuisines with None values if not found
        for i in range(3):
            breakfast_cuisines[i].append(None)

    # Append the hotel name to the list
    hotel_names.append(name_text)

    # Increment the counter of processed URLs
    urls_scraped += 1

    # Print the progress
    print(f"Processed {urls_scraped}/{total_urls_to_scrape} URLs")

# Create a DataFrame
data = {"Hotel Name": hotel_names}
data.update(amenity_availability)
for i in range(3):
    data[f"Breakfast Cuisine {i + 1}"] = breakfast_cuisines[i]

df_Toronto = pd.DataFrame(data)

# Now df is a DataFrame with Hotel Name, the availability of multiple amenities, and Breakfast Cuisine columns
print(df_Toronto)

Processed 1/200 URLs
Processed 2/200 URLs
Processed 3/200 URLs
Processed 4/200 URLs
Processed 5/200 URLs
Processed 6/200 URLs
Processed 7/200 URLs
Processed 8/200 URLs
Processed 9/200 URLs
Processed 10/200 URLs
Processed 11/200 URLs
Processed 12/200 URLs
Processed 13/200 URLs
Processed 14/200 URLs
Processed 15/200 URLs
Processed 16/200 URLs
Processed 17/200 URLs
Processed 18/200 URLs
Processed 19/200 URLs
Processed 20/200 URLs
Processed 21/200 URLs
Processed 22/200 URLs
Processed 23/200 URLs
Processed 24/200 URLs
Processed 25/200 URLs
Processed 26/200 URLs
Processed 27/200 URLs
Processed 28/200 URLs
Processed 29/200 URLs
Processed 30/200 URLs
Processed 31/200 URLs
Processed 32/200 URLs
Processed 33/200 URLs
Processed 34/200 URLs
Processed 35/200 URLs
Processed 36/200 URLs
Processed 37/200 URLs
Processed 38/200 URLs
Processed 39/200 URLs
Processed 40/200 URLs
Processed 41/200 URLs
Processed 42/200 URLs
Processed 43/200 URLs
Processed 44/200 URLs
Processed 45/200 URLs
Processed 46/200 UR

In [127]:
df_Toronto

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,GLOBALSTAY Modern Downtown Apartment,yes,no,yes,no,no,yes,yes,None,None,None
1,Modern Heritage Home - Distillery District,yes,no,no,no,no,no,no,None,None,None
2,STB-Double BED with Private Washroom,yes,no,no,no,no,yes,no,None,None,None
3,Central 1BR & Office - Downtown & 5 Mins to UofT!,yes,no,no,no,no,yes,no,None,None,None
4,Stylish Home with Private Backyard Parking and...,yes,no,no,no,no,yes,no,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
195,Toronto Don Valley Hotel and Suites,no,no,yes,no,no,yes,yes,None,None,None
196,Lilen's Home,yes,no,yes,no,no,yes,no,None,None,None
197,Red Maple Guest House,yes,no,yes,no,no,yes,no,None,None,None
198,Moden California Style 3 Bed 2 Bath with Parki...,yes,no,yes,no,no,no,no,None,None,None


In [128]:
combined_df = pd.concat([df_Dubai, df_london, df_Malaysia, df_Manchester, df_Delhi, df_Birmingham, df_Berlin,
                        df_Sydney, df_Sydney1, df_Paris, df_Tokyo, df_Toronto], ignore_index=True)
combined_df.to_csv('csv2.csv', index=False)

In [129]:
combined_df

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
0,Luxury residence in 5 star hotel,yes,yes,yes,no,yes,no,no,None,None,None
1,Resivation Hotel,yes,no,yes,yes,no,no,no,Continental,Full English/Irish,Vegetarian
2,ibis Styles Dubai Airport Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Vegan
3,Millennium Place Barsha Heights Hotel,yes,yes,yes,no,no,no,no,Continental,Vegetarian,Halal
4,IntercityHotel Dubai Jaddaf Waterfront,yes,yes,yes,no,no,no,no,Continental,Gluten-free,Buffet
...,...,...,...,...,...,...,...,...,...,...,...
2170,Toronto Don Valley Hotel and Suites,no,no,yes,no,no,yes,yes,None,None,None
2171,Lilen's Home,yes,no,yes,no,no,yes,no,None,None,None
2172,Red Maple Guest House,yes,no,yes,no,no,yes,no,None,None,None
2173,Moden California Style 3 Bed 2 Bath with Parki...,yes,no,yes,no,no,no,no,None,None,None


In [130]:
combined_df.tail()

,Hotel Name,Free WiFi,Family rooms,Non-smoking rooms,Restaurant,Bar,Heating,Lift,Breakfast Cuisine 1,Breakfast Cuisine 2,Breakfast Cuisine 3
2170,Toronto Don Valley Hotel and Suites,no,no,yes,no,no,yes,yes,None,None,None
2171,Lilen's Home,yes,no,yes,no,no,yes,no,None,None,None
2172,Red Maple Guest House,yes,no,yes,no,no,yes,no,None,None,None
2173,Moden California Style 3 Bed 2 Bath with Parki...,yes,no,yes,no,no,no,no,None,None,None
2174,"Room in a House in Etobicoke, Free WiFi, Parking",yes,no,no,no,no,yes,no,None,None,None
